# TESTEO OPERADOR $\hat{Q}$

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [ ]:
%load_ext qat.core.magic

In [ ]:
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()    

In [ ]:
from AuxiliarFunctions import  get_histogram, PostProcessResults, TestBins, RunJob
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 6
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

In [ ]:
from QuantumMultiplexors_Module_examples import ExpectationLoadingData
from PhaseAmplification_Module import Load_Q_Gate

## Carga inicial de datos

Despues de la carga de $p(x)$ y $f(x)$ el estado que obtenemos es:

$$|\Psi\rangle=\sqrt{a}|\Psi_{1}\rangle+\sqrt{1-a}|\Psi_{0}\rangle$$

Que se puede poner como:

$$|\Psi\rangle=\sin{\theta}|\Psi_{1}\rangle+\cos{\theta}|\Psi_{0}\rangle$$

siendo $|\Psi_{0}\rangle$ y $|\Psi_{1}\rangle$ dos vectores ortonormales del siguiente modo:

$$|\Psi_{1}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle^{n}\otimes\frac{1}{\sqrt{a}}\sqrt{p(x)f(x)}|1\rangle$$
$$|\Psi_{0}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle^{n}\otimes\frac{1}{\sqrt{1-a}}\sqrt{p(x)(1-f(x))}|0\rangle$$

La siguiente celda implementa esta primera parte

In [ ]:
#Initial State
Qprog, P_Gate, R_gate = ExpectationLoadingData(p_X, f_X)
circuit = Qprog.to_circ()
job = circuit.to_job()
result = RunJob(lineal_qpu.submit(job))
Phi = PostProcessResults(result)

In [ ]:
Phi.head()

A continuación aplicamos el operador:

$$\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

sobre el estado $|\Psi\rangle$. Sabemos este operador ejecuta una rotación de $2\theta$ sobre dicho estado:

$$\hat{Q}|\Psi\rangle= \hat{R}_{y}(2*\theta)|\Psi\rangle=\hat{R}_{y}(2*\theta)(\sin{\theta}|\Psi_{1}\rangle+\cos{\theta}|\Psi_{0}\rangle) = \sin{3\theta}\Psi_{1}\rangle+\cos{3\theta}|\Psi_{0}\rangle$$

En la siguiente celda terminamos de aplicar el circuito:

In [ ]:
qbits = Qprog.registers
Q_Gate = Load_Q_Gate(P_Gate, R_gate)
Qprog.apply(Q_Gate, qbits)
circuit = Qprog.to_circ()
job = circuit.to_job()
result = RunJob(lineal_qpu.submit(job))
Q_Phi= PostProcessResults(result)

In [ ]:
Q_Phi.head()

Asi pues en **Phi** tenemos toda la información del  estado $|\Psi\rangle$ mientras en **Q_Phi** la info del estado $\hat{Q}|\Psi\rangle$. Para comprobar que lo ha hecho bien se debe verificar que:

* $|\Psi\rangle=\sin{\theta}|\Psi_{1}\rangle+\cos{\theta}|\Psi_{0}\rangle$
* $\hat{Q}|\Psi\rangle = \sin{3\theta}\Psi_{1}\rangle+\cos{3\theta}|\Psi_{0}\rangle$

Para ello voy a utilizar la funcion **Get2DVector** que recibe un estado cuántico y devuelve los dos vectores ortonormales: $|\Psi_{0}\rangle$ y $|\Psi_{1}\rangle$ y sus amplitudes correspondientes. Es decir:
1. Si le damos el estado **Phi**: obtendremos: $|\Psi_{0}\rangle$ y $|\Psi_{1}\rangle$ y ($\cos{\theta}, \sin{\theta}$)
2. Si le damos el estado **QPhi** deberíamos obtener:  $|\Psi_{0}\rangle$ y $|\Psi_{1}\rangle$ y ($\cos{3\theta}, \sin{3\theta}$)

Vamos a ello:

In [ ]:
def Normalize(pds):
    Normalization = sum(pds**2)
    return pds/np.sqrt(Normalization), np.sqrt(Normalization)

def Get2DVector(pdf):
    Phi_0 = pdf.iloc[[i for i in range(0, len(pdf), 2)]]['Amplitude']
    Phi_0.reset_index(inplace=True, drop=True)
    Phi_0.rename('|0>', inplace = True)
    Phi_1 = pdf.iloc[[i for i in range(1, len(pdf), 2)]]['Amplitude']
    Phi_1.reset_index(inplace=True, drop=True)
    Phi_1.rename('|1>', inplace = True)
    NPhi_0, N0 = Normalize(Phi_0)
    NPhi_1, N1 = Normalize(Phi_1)
    #Suma = Phi_0 + Phi_1
    return pd.concat([NPhi_0, NPhi_1], axis=1), np.array([N0, N1])

In [ ]:
Phi_0, alfas= Get2DVector(Phi)
QPhi_0, Qalfas= Get2DVector(Q_Phi)

Vamos a ver que ocurre con los vectores $|\Psi_{0}\rangle$ y $|\Psi_{1}\rangle$ obtenidos para **Phi** (pandas dataframe *Phi_0*) y para **Q_Phi** (pandas dataframe *QPhi_0*)

In [ ]:
Phi_0.head()

In [ ]:
QPhi_0.head()

Como se puede observar los $|\Psi_{1}\rangle$ obtenidos para **Phi** y para **Q_Phi** son exactamente iguales mientras que los $|\Psi_{0}\rangle$ son opuestos. Vamos a cambiar el signo de estos últimos: se lo quitmaos a los vectores y se lo ponemos a las amplitudes:

In [ ]:
QPhi_0['|0>']=-QPhi_0['|0>']
Qalfas[0] = -Qalfas[0]

Con estos cambios tenemos todo alineado. Lo que vamos a obtener son los angulos $\theta$ de las amplitudes correspondientes a **Phi** (*alfas*) y a **QPhi** (*Qalfas*) y comprobaremos que  el segundo es tres veces el primero como tiene que ser!!!

Para el cos (primera coordenada de *alfas* y *Qalfas* esta comprobación es directa!!

In [ ]:
print('El angulo de Qalfas 3.0 veces el alfas: {}'.format(np.isclose(np.arccos(Qalfas[0])/3.0, np.arccos(alfas[0]))))

Para el caso del *seno* (segunda coordenada de *alfas* y *Qalfas*) la cosa es un poco ás enrevesada por como las funciones trigonométricas de numpy funcionan (y porque hay que recordar que sen(PI-alfa) = sen(alfa)

In [ ]:
np.isclose(np.arcsin(Qalfas[0])/3.0, np.arcsin(alfas[0]))

In [ ]:
np.arcsin(Qalfas[0])

In [ ]:
Beta = np.arcsin(Qalfas[0])

In [ ]:
#Como el seno es negativo tenemos que buscar el tercer cuadrante en vez del segundo!!
Pi_MenosBeta = 3*np.arcsin(alfas[0])- np.pi 

In [ ]:
np.isclose(Beta, Pi_MenosBeta)